In [7]:
import mediapipe as mp
import numpy as np
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [8]:
# Use OpenCV’s VideoCapture to start capturing from the webcam.
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

: 

In [ ]:
model_path = './models/blaze_face_short_range.tflite'

BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
FaceDetectorResult = mp.tasks.vision.FaceDetectorResult
VisionRunningMode = mp.tasks.vision.RunningMode

class storage:
    def __init__(self):
        self.image = np.zeros((480, 640), dtype=np.uint8)

    def get_annotated_image(self):
        return self.image
    
    def save_annotated_image(self, img):
        self.image = img

# 이미지 저장하는 클래스
image_storage = storage()

In [ ]:
# Create a face detector instance with the live stream mode:
def print_result(result: FaceDetectorResult, output_image: mp.Image, timestamp_ms: int):
    print('face detector result: {}'.format(result))

options = FaceDetectorOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

In [ ]:
with FaceDetector.create_from_options(options) as detector:
    # The detector is initialized. Use it here.

    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while webcam.isOpened():
        status, frame = webcam.read()
        frame_timestamp_ms = int(webcam.get(cv2.CAP_PROP_POS_MSEC))

        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            
        # Send live image data to perform face detection.
        # The results are accessible via the `result_callback` provided in
        # the `FaceDetectorOptions` object.
        # The face detector must be created with the live stream mode.
        detector.detect_async(mp_image, frame_timestamp_ms)
            
        if status:
            cv2.imshow("test", image_storage.get_annotated_image())
            #cv2.imshow("test", annotated_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cv2.destroyAllWindows()

    # Convert the frame received from OpenCV to a MediaPipe’s Image object.

face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detections=[])
face detector result: DetectionResult(detectio

KeyboardInterrupt: 